In [ ]:
import seaborn as sns
import pandas as pd

a = pd.read_csv("dataSets/dat-ab-usos-2024.csv")
 
amba = a[(a["AMBA"] == "SI") & (a["PROVINCIA"] == "BUENOS AIRES")]

print(amba["NOMBRE_EMPRESA"].nunique())